In [1]:
import torch
from torch import nn 
import numpy as np
from datasets import load_dataset, load_from_disk
from huggingface_hub import HfFolder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
torch.cuda.is_available()

c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [ ]:
HF_TOKEN = "hf..."
HfFolder.save_token(HF_TOKEN)
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [10]:
ds = load_dataset("ucirvine/sms_spam")
ds.save_to_disk("E:/2.code-on-fire/3.ML/Sms-Spam/dataset")

Saving the dataset (1/1 shards): 100%|██████████| 5574/5574 [00:00<00:00, 501804.05 examples/s]


In [3]:
ds = load_from_disk("E:/2.code-on-fire/3.ML/Sms-Spam/dataset")

In [5]:
ds["train"][:0]["sms"]

[]

Data Preperation

In [4]:
cv = CountVectorizer(dtype=np.float32)
X = cv.fit_transform(ds["train"][0:]["sms"])

In [5]:
X = X.toarray()
X = torch.from_numpy(X).type(torch.float)
y= np.array(ds["train"]["label"])
y = torch.from_numpy(y).type(torch.float)
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2,
                                                    random_state=42) 

In [6]:
print(X_train.shape[0] ,X_test.shape[0] ,len(y_train) ,len(y_test))

4459 1115 4459 1115


MODEL

In [7]:
class CircleModelV0(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(8713, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        return self.model(x)

model_0 = CircleModelV0().to(device)
model_0

CircleModelV0(
  (model): Sequential(
    (0): Linear(in_features=8713, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [12]:
# Setup
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() 
    acc = (correct / len(y_pred)) * 100 
    return acc

y_logits = model_0(X_test.to(device))[:5]
y_pred_probs = torch.sigmoid(y_logits)
y_preds = torch.round(y_pred_probs)
y_pred_labels = torch.round(torch.sigmoid(model_0(X_test.to(device))[:5]))
print(torch.eq(y_preds.squeeze(), y_pred_labels.squeeze()))
y_preds.squeeze()
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_0.parameters(), lr=0.001)

tensor([True, True, True, True, True], device='cuda:0')


Traning and testinggg loop

In [13]:
torch.manual_seed(42)


epochs = 1000

# Put data to target device
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

# Build training and evaluation loop
for epoch in range(epochs):
    ### Training
    model_0.train()

    # 1. Forward pass
    y_logits = model_0(X_train).squeeze() 
    y_pred = torch.round(torch.sigmoid(y_logits)) 
  
    # 2. Calculate loss/accuracy
    loss = loss_fn(y_logits,
                   y_train) 
    acc = accuracy_fn(y_true=y_train, 
                      y_pred=y_pred) 

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model_0.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_logits = model_0(X_test).squeeze() 
        test_pred = torch.round(torch.sigmoid(test_logits))
        # 2. Caculate loss/accuracy
        test_loss = loss_fn(test_logits,
                            y_test)
        test_acc = accuracy_fn(y_true=y_test,
                               y_pred=test_pred)

    # Print out what's happening every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.67886, Accuracy: 86.70% | Test loss: 0.66974, Test acc: 86.01%
Epoch: 10 | Loss: 0.56483, Accuracy: 97.78% | Test loss: 0.56147, Test acc: 96.68%
Epoch: 20 | Loss: 0.43694, Accuracy: 99.08% | Test loss: 0.44372, Test acc: 97.76%
Epoch: 30 | Loss: 0.32340, Accuracy: 99.35% | Test loss: 0.34034, Test acc: 98.12%
Epoch: 40 | Loss: 0.23549, Accuracy: 99.55% | Test loss: 0.26125, Test acc: 98.12%
Epoch: 50 | Loss: 0.17286, Accuracy: 99.64% | Test loss: 0.20590, Test acc: 98.12%
Epoch: 60 | Loss: 0.12957, Accuracy: 99.71% | Test loss: 0.16871, Test acc: 98.12%
Epoch: 70 | Loss: 0.09960, Accuracy: 99.82% | Test loss: 0.14387, Test acc: 98.21%
Epoch: 80 | Loss: 0.07851, Accuracy: 99.91% | Test loss: 0.12708, Test acc: 98.03%
Epoch: 90 | Loss: 0.06333, Accuracy: 99.91% | Test loss: 0.11549, Test acc: 98.12%
Epoch: 100 | Loss: 0.05212, Accuracy: 99.93% | Test loss: 0.10733, Test acc: 98.12%
Epoch: 110 | Loss: 0.04364, Accuracy: 99.93% | Test loss: 0.10148, Test acc: 98.12%
Epo

In [14]:
def ped(text, vectorizer, model):
    if isinstance(text, str):
        text = [text]
    X = vectorizer.transform(text)
    X = torch.tensor(X.toarray(), dtype=torch.float32).to(next(model.parameters()).device)
    model.eval()
    with torch.inference_mode():
        logits = model(X).squeeze()
        probs = torch.sigmoid(logits)
        preds = (probs >= 0.5).long()
    return preds


In [15]:
text1 = "hiii!"
text2 = "England v Macedonia - dont miss the goals/team news. Txt ur national team to 87077 eg ENGLAND to 87077 Try:WALES, SCOTLAND 4txt/ú1.20 POBOXox36504W45WQ 16+"
result =  ped(text1,cv,model_0)
print("Spam" if result.item() == 1 else "Not spam")

Not spam


Saving the model

In [16]:
## Save model
# torch.save(model_0.state_dict(), "model_weights.pth")
torch.save(model_0, "full_model.pth")

In [ ]:
## Load model weights
# model_0 = CircleModelV0().to(device)
# model_0.load_state_dict(torch.load("model_weights.pth"))
# model_0.eval()
# FULL MODEL
model_0 = torch.load("full_model.pth")
model_0.eval()

In [17]:
import pickle
with open("vectorizer.pkl", "wb") as f:
    pickle.dump(cv, f)